In [ ]:
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow
import emission.core.get_database as edb
from emission.core.wrapper.user import User
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import emission.storage.timeseries.abstract_timeseries as esta
from datetime import timedelta, date, tzinfo, datetime
import numpy as np

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get('2018-03-28', 'YYYY-MM-DD').timestamp, arrow.get('2018-06-06', 'YYYY-MM-DD').timestamp)
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)
client_nav_events_df.head()

In [ ]:
opened_app_events = client_nav_events_df[client_nav_events_df.name == "opened_app"]

# Permutation Tests

In [ ]:
users = edb.get_uuid_db().find()

In [ ]:
# Create a dataframe with columns user_id, , number notifications opened, week number, and group.
df = pd.DataFrame()
information_count = 0
emotion_count = 0
control_count = 0
for i in range(len(users)):
    user_id = users[i]
    start = arrow.get('2018-03-28', 'YYYY-MM-DD')
    end = arrow.get('2018-06-06', 'YYYY-MM-DD')
    vals = []
    week_val = -1
    for week in arrow.Arrow.range('week', start, end):
        ts = esta.TimeSeries.get_time_series(user_id)
        begin_ts = week.timestamp
        end_ts = (week + timedelta(weeks=1)).timestamp
        last_period_tq = estt.TimeQuery("data.start_ts", begin_ts, end_ts)
        cs_df = ts.get_data_df("analysis/inferred_section", time_query=last_period_tq)
        total = 0
        if cs_df.shape[0] <= 0:
            continue
        try:
            for event in opened_app_events:
                if event['user_id'] == user_id:
                    if event['ts'] > begin_ts and event['ts'] <= end_ts:
                        total += 1
        except:
            continue
        vals.append(total)
    #Always use lists only where the number of datapoints is greater than 2 otherwise we get a perfect correlation
    weeks = np.arange(len(vals))
    if len(weeks) > 1:
        group = "none"
        try:
            client = edb.get_profile_db().find_one({"user_id": user_id})['client']
            if client == 'urap-2017-information':
                group = "information"
                information_count += 1
            elif client == 'urap-2017-emotion':
                group = "emotion"
                emotion_count += 1
            elif client == 'urap-2017-control':
                group = "control"
                control_count += 1
        except:
            continue
        df = df.append({'uuid': user_id, 'group': group, 'total': sum(vals)}, ignore_index=True)

In [ ]:
df.groupby('group').mean()

In [ ]:
print(information_count, emotion_count, control_count)

In [ ]:
df.head()

In [ ]:
df.groupby('group').mean()

In [ ]:
def mean_diff(vals_0, vals_1):
    return np.mean(vals_0) - np.mean(vals_1)

In [ ]:
def perm_test(labels, response_vars, stat_func, n):
    '''Labels: Series with two labels, Response_vars series in same order as labels
    stat_func is a function that takes in two series and returns a statistic, n is permutation numnber'''
    unique_label_counts = labels.value_counts()
    label_0 = unique_label_counts.index[0]
    label_1 = unique_label_counts.index[1]
    label_0_count = unique_label_counts[0]
    label_1_count = unique_label_counts[1]
    vals_0 = response_vars[labels == label_0]
    vals_1 = response_vars[labels == label_1]
    observed_stat = stat_func(vals_0, vals_1)
    sample_stats = np.array([])
    ind = labels
    for i in range(n):
        sampler = np.random.permutation(label_0_count + label_1_count)
        new_vals = response_vars.take(sampler).values
        df = pd.DataFrame({'vals': new_vals}, index=ind)
        vals_0 = df[df.index == label_0]['vals']
        vals_1 = df[df.index == label_1]['vals']
        stat = stat_func(vals_0, vals_1)
        sample_stats = np.append(sample_stats, stat)
    perm_mean = np.mean(sample_stats)
    plt.hist(sample_stats)
    plt.show()
    if observed_stat > perm_mean:
        return np.sum(sample_stats > observed_stat) / len(sample_stats)
    return np.sum(sample_stats < observed_stat) / len(sample_stats)

In [ ]:
from math import sqrt
def print_error_percent(p, n):
    print("p value: ", p)
    print("error percent: {0}%".format(sqrt(p * (1-p) / n) * 2 * 100))

In [ ]:
e_c = df[df['group'] != 'information']
print_error_percent(perm_test(e_c['group'], e_c['total'], mean_diff, 100000), 100000)

In [ ]:
i_c = df[df['group'] != 'emotion']
print_error_percent(perm_test(i_c['group'], i_c['total'], mean_diff, 100000), 100000)

In [ ]:
i_e = df[df['group'] != 'control']
print_error_percent(perm_test(i_e['group'], i_e['total'], mean_diff, 100000), 100000)

# Bootstrapping Tests

In [ ]:
# Same as permutation testing but sampling is with replacement.
# Also don't include iteration if SD's of both groups are 0.

def bootstrap_test(labels, response_vars, stat_func, n):
    '''Labels: Series with two labels, Response_vars series in same order as labels
    stat_func is a function that takes in two series and returns a statistic, n is permutation numnber'''
    unique_label_counts = labels.value_counts()
    label_0 = unique_label_counts.index[0]
    label_1 = unique_label_counts.index[1]
    label_0_count = unique_label_counts[0]
    label_1_count = unique_label_counts[1]
    vals_0 = response_vars[labels == label_0]
    vals_1 = response_vars[labels == label_1]
    observed_stat = stat_func(vals_0, vals_1)
    sample_stats = np.array([])
    ind = labels
    for i in range(n):
        sampler = np.random.choice(np.random.permutation(label_0_count + label_1_count), label_0_count + label_1_count)
        new_vals = response_vars.take(sampler).values
        df = pd.DataFrame({'vals': new_vals}, index=ind)
        vals_0 = df[df.index == label_0]['vals']
        vals_1 = df[df.index == label_1]['vals']
        if np.std(vals_0) == 0 and np.std(vals_1) == 0:
            continue
        stat = stat_func(vals_0, vals_1)
        sample_stats = np.append(sample_stats, stat)
    perm_mean = np.mean(sample_stats)
    plt.hist(sample_stats)
    plt.show()
    if observed_stat > perm_mean:
        return np.sum(sample_stats > observed_stat) / len(sample_stats)
    return np.sum(sample_stats < observed_stat) / len(sample_stats)

In [ ]:
e_c = df[df['group'] != 'information']
print_error_percent(bootstrap_test(e_c['group'], e_c['total'], mean_diff, 100000), 100000)

In [ ]:
i_c = df[df['group'] != 'emotion']
print_error_percent(bootstrap_test(i_c['group'], i_c['total'], mean_diff, 100000), 100000)

# Mann Whitney U Tests

In [ ]:
from scipy.stats import mannwhitneyu

control = df[df['group'] == 'control']
control_array = control.as_matrix(columns=control.columns[1:2])

info = df[df['group'] == 'information']
info_array = info.as_matrix(columns=info.columns[1:2])

emotion = df[df['group'] == 'emotion']
emotion_array = emotion.as_matrix(columns=emotion.columns[1:2])

print(mannwhitneyu(info_array, control_array))
print(mannwhitneyu(emotion_array, control_array))